# Prenight Briefing for {{ params.day_obs }}

This is presently just a playground for building a pre-night briefing.

In [ ]:
# This cell is only for setting parameter defaults
day_obs = "2024-09-04"
sim_date = "2024-09-04"

In [ ]:
# Validate the inputs
import re
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', day_obs) is not None
assert re.match(r'^\d\d\d\d-\d\d-\d\d$', sim_date) is not None

In [ ]:
from IPython.display import display, HTML
import datetime
import sys
import os
import yaml
from urllib.parse import urlparse
import warnings
import pandas as pd
import numpy as np
import astropy
import bokeh
import bokeh.io
import boto3
import colorcet
from erfa import ErfaWarning
from astropy.time import Time

In [ ]:
os.environ['RUBIN_SIM_DATA_DIR'] = '/sdf/data/rubin/user/neilsen/data/rubin_sim_data'

In [ ]:
sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/pip_targets/lib/python3.11/site-packages')

In [ ]:
devel_versions = True
if devel_versions:
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/times_square_sources/2024-03-25/uranography')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_scheduler')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/rubin_sim')
    sys.path.insert(0, '/sdf/data/rubin/user/neilsen/devel/schedview')

In [ ]:
import rubin_scheduler
import rubin_scheduler.utils
import rubin_scheduler.sim_archive
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_sim import maf
from lsst.resources import ResourcePath

In [ ]:
import schedview.compute
import schedview.compute.visits
import schedview.collect
import schedview.collect.rewards
import schedview.plot
import schedview.plot.rewards

In [ ]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

In [ ]:
# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=ErfaWarning,
    message=r".*dubious year.*",
)

In [ ]:
# Don't complain about working with daytime MJDs either.
warnings.filterwarnings(
    "ignore",
    module="rubin_scheduler.skybrightness_pre.sky_model_pre",
    category=UserWarning,
    message="Requested MJD between sunrise and sunset, returning closest maps",
)

In [ ]:
# Quiet unimportant chatter from healpy.
healpy_logger = logging.getLogger("healpy")
healpy_logger.setLevel(logging.WARNING)

In [ ]:
bokeh.io.output_notebook()

In [ ]:
%matplotlib inline

In [ ]:
archive_uri = "s3://rubin:rubin-scheduler-prenight/opsim/"

if urlparse(archive_uri).scheme.upper() == 'S3':
    os.environ["LSST_DISABLE_BUCKET_VALIDATION"] = "1"
    os.environ["S3_ENDPOINT_URL"] = "https://s3dfrgw.slac.stanford.edu/"

In [ ]:
day_obs_mjd = int(Time(day_obs).mjd)
observatory = ModelObservatory(init_load_length=1)
timezone = "Chile/Continental"

## Astronomical events during the night

In [ ]:
day_obs_datetime = Time(day_obs_mjd, format='mjd').datetime
day_obs_date = datetime.date(day_obs_datetime.year, day_obs_datetime.month, day_obs_datetime.day)
night_events = schedview.compute.astro.night_events(day_obs_date)
night_events

## Sun and moon positions in the middle of the night

In [ ]:
model_observatory = ModelObservatory(init_load_length=1)
model_observatory.mjd = night_events.loc['night_middle', 'MJD']

In [ ]:
body_positions_wide = pd.DataFrame(model_observatory.almanac.get_sun_moon_positions(night_events.loc['night_middle', 'MJD']))
body_positions_wide.index.name = 'r'
body_positions_wide.reset_index(inplace=True)

angle_columns = ['RA', 'dec', 'alt', 'az']
all_columns = angle_columns + ['phase']
body_positions = (
    pd.wide_to_long(body_positions_wide, stubnames=('sun', 'moon'), suffix=r'.*', sep='_', i='r', j='')
    .droplevel('r')
    .T[all_columns]
)
body_positions[angle_columns] = np.degrees(body_positions[angle_columns])
body_positions

All angles are in degrees.

## Simulated visits

In [ ]:
sims_metadata = rubin_scheduler.sim_archive.read_archived_sim_metadata(archive_uri, latest=sim_date, num_nights=1)

In [ ]:
visit_columns = [
    "observationId",
    "fieldRA",
    "fieldDec",
    "observationStartMJD",
    "flush_by_mjd",
    "visitExposureTime",
    "filter",
    "rotSkyPos",
    "rotSkyPos_desired",
    "numExposures",
    "airmass",
    "seeingFwhm500",
    "seeingFwhmEff",
    "seeingFwhmGeom",
    "skyBrightness",
    "night",
    "slewTime",
    "visitTime",
    "slewDistance",
    "fiveSigmaDepth",
    "altitude",
    "azimuth",
    "paraAngle",
    "cloud",
    "moonAlt",
    "sunAlt",
    "note",
    "block_id",
    "observationStartLST",
    "rotTelPos",
    "rotTelPos_backup",
    "moonAz",
    "sunAz",
    "sunRA",
    "sunDec",
    "moonRA",
    "moonDec",
    "moonDistance",
    "solarElong",
    "moonPhase",
    "cummTelAz",
    "scripted_id",
]

In [ ]:
visits_list = []
for sim_uri, sim_metadata in sims_metadata.items():
    first_day_obs_mjd = astropy.time.Time(sim_metadata['simulated_dates']['first']).mjd
    last_day_obs_mjd = astropy.time.Time(sim_metadata['simulated_dates']['last']).mjd

    includes_day_obs = first_day_obs_mjd <= day_obs_mjd <= last_day_obs_mjd

    if not includes_day_obs:
        continue

    sim_rp = ResourcePath(sim_uri).join(sim_metadata['files']['observations']['name'])

    these_visits = schedview.collect.read_opsim(
        sim_rp,
        constraint=f"FLOOR(observationStartMJD-0.5)={day_obs_mjd}",
        stackers=[
            maf.stackers.TeffStacker(),
            maf.stackers.ObservationStartDatetime64Stacker(),
            maf.stackers.DayObsStacker(),
            maf.stackers.DayObsMJDStacker(),
            maf.stackers.DayObsISOStacker(),
            maf.stackers.OverheadStacker()
        ],
        dbcols=visit_columns
    )
    these_visits = schedview.compute.visits.add_coords_tuple(these_visits)

    for key in ['label', 'opsim_config_branch', 'opsim_config_repository', 'opsim_config_script', 'scheduler_version', 'sim_runner_kwargs', 'tags']:
        these_visits[key] = [sim_metadata[key]] * len(these_visits)

    visits_list.append(these_visits)

visits = pd.concat(visits_list)
visits_ds = bokeh.models.ColumnDataSource(visits)

In [ ]:
sim_labels = visits['label'].unique()
num_sims = len(sim_labels)
sim_color_mapper = bokeh.models.CategoricalColorMapper(
    factors=sim_labels, palette=colorcet.palette["glasbey"][: num_sims], name="simulation"
)

# Some bokeh symbols have the same outer shape but different inner markings,
# but these are harder to distinguish, so put them at the end.
all_markers = [m for m in bokeh.core.enums.MarkerType if "_" not in m] + [m for m in bokeh.core.enums.MarkerType if "_" in m]

# dot is hard to see
all_markers.remove('dot')
sim_marker_mapper = bokeh.models.CategoricalMarkerMapper(
    factors=sim_labels,
    markers=all_markers[:num_sims],
    name="simulation",
)

In [ ]:
fig = bokeh.plotting.figure(
    title="Altitude",
    x_axis_label="Time (UTC)",
    y_axis_label="Altitude",
    frame_width=1536,
    frame_height=512,
)

fig.scatter(x='start_date',
            y='altitude',
            legend_group='label',
            fill_alpha=0.2,
            color={"field": "label", "transform": sim_color_mapper},
            marker={"field": "label", "transform": sim_marker_mapper},
            source=visits_ds)
bokeh.io.show(fig)